In [43]:
import pandas as pd
import numpy as np
import holoviews as hv
import panel as pn
import networkx as nx
import requests
import json
from holoviews.ipython import display

### SDG Data API

In [14]:
def all_geoareas():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/GeoArea/List")
    return pd.DataFrame(data.json())
    
def all_goals():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=false")
    return pd.DataFrame(data.json())

def all_targets():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Target/List?includechildren=true")
    return pd.DataFrame(data.json())

def all_indicators():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Indicator/List")
    return pd.DataFrame(data.json())

def all_series():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Series/List")
    return pd.DataFrame(data.json())

In [3]:
ecosystem_capitals = pd.DataFrame([
            'Natural Capital', 
            'Intellectual Capital', 
            'Human Capital', 
            'Manufactured Capital', 
            'Financial Capital', 
            'Social & Relationship Capial',
        ],columns=['Ecosystem Capitals'])

In [15]:
all_geoareas = all_geoareas()
all_goals = all_goals()
all_targets = all_targets()
all_indicators = all_indicators()
all_series = all_series()

In [ ]:
geo_data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/GeoArea/Tree")
geo_tree = geo_data.json()

sdg_data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=true")
sdg_data = pd.DataFrame(sdg_data.json())

### GeoTree

In [727]:
def traverse(G, v=None):
    for n in G:
        nodes_data.append({'name':n['geoAreaName'],'group':n['type'],'code':n['geoAreaCode']})
        if v:
            links_data.append({'value':1,'source':v['geoAreaCode'],'target':n['geoAreaCode']})
        if n['children']:
            traverse(n['children'], n)

In [728]:
df = pd.DataFrame(geo_tree)
all_geos = []

In [729]:
def traverse(node, tree):
    if len(tree):
        tree['Parent'] = node
        all_geos.append(tree)
        children = tree['children'].apply(pd.DataFrame)
        [traverse(n, t) for n, t in zip(tree['geoAreaCode'], children)]

In [730]:
traverse(1, pd.DataFrame(df.iloc[0]['children']))

In [731]:
df = pd.concat(all_geos)

In [732]:
df

,geoAreaCode,geoAreaName,type,children,Parent
0,10,Antarctica,Country,None,1
1,2,Africa,Region,"[{'geoAreaCode': 15, 'geoAreaName': 'Northern ...",1
2,19,Americas,Region,"[{'geoAreaCode': 419, 'geoAreaName': 'Latin Am...",1
3,150,Europe,Region,"[{'geoAreaCode': 151, 'geoAreaName': 'Eastern ...",1
4,142,Asia,Region,"[{'geoAreaCode': 143, 'geoAreaName': 'Central ...",1
...,...,...,...,...,...
5,882,Samoa,Country,None,61
6,772,Tokelau,Country,None,61
7,776,Tonga,Country,None,61
8,798,Tuvalu,Country,None,61


In [821]:
geo_nodes = df[['geoAreaName','type','geoAreaCode']].copy()
geo_nodes.columns = ['name','group','code']
geo_nodes['size'] = 3
geo_nodes['code'] = geo_nodes['code'].apply(lambda x: 'g'+str(x))
geo_nodes = pd.concat([geo_nodes,pd.DataFrame([['World','World','g1','10']],columns=geo_nodes.columns)],ignore_index=True)

In [822]:
geo_links = df[['Parent','geoAreaCode']].copy()
geo_links['value'] = 1
geo_links = geo_links.rename({'Parent':'source','geoAreaCode':'target'},axis=1)[['value','source','target']]
geo_links['source'] = geo_links['source'].apply(lambda x: 'g'+str(x))
geo_links['target'] = geo_links['target'].apply(lambda x: 'g'+str(x))
geo_links = geo_links.replace('g0','g1').iloc[1:]

In [823]:
geo_nodes

,name,group,code,size
0,Antarctica,Country,g10,3
1,Africa,Region,g2,3
2,Americas,Region,g19,3
3,Europe,Region,g150,3
4,Asia,Region,g142,3
...,...,...,...,...
291,Tokelau,Country,g772,3
292,Tonga,Country,g776,3
293,Tuvalu,Country,g798,3
294,Wallis and Futuna Islands,Country,g876,3


In [824]:
# geo_links = geo_links[~geo_links['target'].isin(['g150','g142','g513','g747','g753'])]

In [825]:
len(geo_links['target'].unique())

294

In [826]:
# links_data = []
# nodes_data = []
# traverse(geo_tree)

In [827]:
# nodes_data

In [828]:
# links_data

### Nodes - Geo - Eco Capitals - SDGs - Zone of Life

In [829]:
# geo_links = pd.DataFrame(links_data)
# geo_links['source'] = geo_links['source'].apply(lambda x: 'g'+str(x))
# geo_links['target'] = geo_links['target'].apply(lambda x: 'g'+str(x))
# geo_nodes = pd.DataFrame(nodes_data)
# geo_nodes['code'] = geo_nodes['code'].apply(lambda x: 'g'+str(x))
# geo_nodes['size'] = 3

In [830]:
geo_nodes

,name,group,code,size
0,Antarctica,Country,g10,3
1,Africa,Region,g2,3
2,Americas,Region,g19,3
3,Europe,Region,g150,3
4,Asia,Region,g142,3
...,...,...,...,...
291,Tokelau,Country,g772,3
292,Tonga,Country,g776,3
293,Tuvalu,Country,g798,3
294,Wallis and Futuna Islands,Country,g876,3


In [831]:
ecosystem_capitals_nodes = pd.DataFrame()
ecosystem_capitals_nodes['name'] = ecosystem_capitals['Ecosystem Capitals']
ecosystem_capitals_nodes['group'] = 'Ecosystem Capitals'
ecosystem_capitals_nodes['code'] = ecosystem_capitals.index

In [832]:
sdg_nodes = pd.DataFrame()
sdg_nodes['name'] = sdg_data['title']
sdg_nodes['group'] = 'SDG'
sdg_nodes['code'] = sdg_data['code']
sdg_nodes['size'] = 5

In [833]:
# zone_of_life_node = pd.DataFrame([['Zone of Life','ZOL',9001, 20]],columns=['name','group','code','size'])

In [834]:
# zone_of_life_node['code'] = zone_of_life_node['code'].apply(lambda x: 'z'+str(x))
sdg_nodes['code'] = sdg_nodes['code'].apply(lambda x: 's'+str(x))
ecosystem_capitals_nodes['code'] = ecosystem_capitals_nodes['code'].apply(lambda x: 'e'+str(x))

### Links - Geo - Eco Capitals - Zone of Life

In [835]:
eco_links1 = pd.DataFrame()
eco_links1['source'] = ecosystem_capitals_nodes['code']
eco_links1['target'] = 'g1'
eco_links1['value'] = '0.75'
# eco_links2 = pd.DataFrame()
# eco_links2['source'] = ecosystem_capitals_nodes['code']
# eco_links2['target'] = 'z9001'
# eco_links2['value'] = '0.75'
# eco_links = pd.concat([eco_links1,eco_links2],sort=False)
                                
sdg_links = pd.DataFrame()
sdg_links['target'] = sdg_nodes['code']
sdg_links['source'] = 'g1'
sdg_links['value'] = '0.25'

In [836]:
nodes_df = pd.concat([geo_nodes,ecosystem_capitals_nodes,sdg_nodes],sort=False).reset_index()
links_df = pd.concat([geo_links,sdg_links,eco_links1,],sort=False)

### Mapping Targets to SDGs

In [838]:
target_nodes = pd.DataFrame()
target_nodes['name'] = all_targets['title']
target_nodes['group'] = 'Targets'
target_nodes['code'] = all_targets['code'].apply(lambda x: 's'+str(x).replace('.','-'))
target_nodes['goal_code'] = all_targets['goal'].apply(lambda x: 's'+str(x).replace('.','-'))
target_nodes['size'] = 3

target_links = pd.DataFrame()
target_links['source'] = target_nodes['code']
target_links['target'] = target_nodes['goal_code']
target_links['value'] = 1

nodes_df = pd.concat([geo_nodes,ecosystem_capitals_nodes,sdg_nodes,target_nodes[['name','group','code','size']]],sort=False).reset_index()
links_df = pd.concat([geo_links,sdg_links,eco_links1,target_links[['value','source','target']]],sort=False)
links_df.loc[len(links_df)] = [0.1,'g1','z9001']

### Map node codes

In [839]:
code_to_index = dict(zip(nodes_df['code'],nodes_df.index))
links_df_mapped = pd.DataFrame()
links_df_mapped['source'] = links_df['source'].map(code_to_index)
links_df_mapped['target'] = links_df['target'].map(code_to_index)

In [840]:
links_df_mapped = links_df_mapped.fillna(1)

### Convert to JSON and dump to disk

In [841]:
links_list = links_df_mapped.to_dict('records')

nodes_list = nodes_df[['name','group']].to_dict('records')

json_prep = {"nodes":nodes_list, "links":links_list}

json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

filename_out = 'pcap_export.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()